<a href="https://colab.research.google.com/github/wfreinhart/sdmm-regression/blob/main/notebooks/generate_histograms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs

Install dependencies.

In [ ]:
try:
    import MDAnalysis
except:
    !pip install MDAnalysis==1.0.0
    !pip install numba==0.51.2
    !pip install umap-learn
    exit()  # need to restart the kernel to access the packages

     |████████████████████████████████| 19.6 MB 1.4 MB/s 
     |████████████████████████████████| 2.3 MB 31.7 MB/s 
     |████████████████████████████████| 2.1 MB 32.6 MB/s 
     |████████████████████████████████| 374 kB 48.1 MB/s 
     |████████████████████████████████| 40 kB 21 kB/s 
  Created wheel for MDAnalysis: filename=MDAnalysis-1.0.0-cp37-cp37m-linux_x86_64.whl size=4549234 sha256=05e7b8cf06a31df7fbe9ba86ac71fa7d1bcb3f94d20c282940ad090f87ae6d1a
  Stored in directory: /root/.cache/pip/wheels/e6/2c/3a/d2b056f1f24662cf00b6a298bf66856addd3fe942d7426717a
Successfully built MDAnalysis
     |████████████████████████████████| 88 kB 3.7 MB/s 
     |████████████████████████████████| 1.1 MB 32.9 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=43471ad1416fca8e450d2b04a93282cc9cd12f52bc4c5dc031dce185ae88c02a
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynnde

# Definitions

Define the atomic environment descriptor model.

In [ ]:
import torch
from torch import nn
import numpy as np


class GaussianHistogram(nn.Module):
    def __init__(self, bins, ranges, sigma, device='cpu'):
        super(GaussianHistogram, self).__init__()

        self.device = device

        # create sigma tensor of appropriate shape
        self.sigma = torch.tensor(sigma.reshape(3, 1, 1), dtype=torch.float, device=torch.device(device))

        # create bin vectors
        self.bins = bins
        rmin = torch.tensor(ranges[:, [0]], dtype=torch.float, device=torch.device(device))
        rmax = torch.tensor(ranges[:, [1]], dtype=torch.float, device=torch.device(device))
        delta = (rmax - rmin) / float(bins)
        self.centers = rmin + delta * (torch.arange(bins, device=torch.device(device)).float() + 0.5)
        self.delta = delta.reshape(3, 1, 1)

        # create centers grid
        self.xy = []
        for i, j in [(0, 1), (0, 2), (1, 2)]:
            xv, yv = torch.meshgrid(self.centers[i], self.centers[j])
            xy = torch.vstack([xv.reshape(1, -1), yv.reshape(1, -1)])
            self.xy.append(torch.unsqueeze(xy, 2))

    def forward(self, x):
        eps = 1e-6

        # generate the histograms
        z = torch.zeros([3, self.bins ** 2], device=self.device)
        for k, ij in enumerate([(0, 1), (0, 2), (1, 2)]):
            # do the gaussian expansion
            y = torch.unsqueeze(x[ij, :], 1) - self.xy[k]
            y = torch.exp(-0.5 * (y / self.sigma[[ij]]) ** 2) / (self.sigma[[ij]] * np.sqrt(np.pi * 2)) * \
                self.delta[[ij]]
            y = y.prod(dim=0)
            z[k] = y.sum(dim=1)

        # normalize
        z /= torch.unsqueeze(z.sum(dim=-1) + eps, -1)

        return z

    def to(self, device, *args, **kwargs):
        super(GaussianHistogram, self).to(*args, **kwargs)
        self.sigma = self.sigma.to(device)
        self.delta = self.delta.to(device)
        self.xy = [x.to(device) for x in self.xy]

Define the coarse graining procedure.

In [ ]:
import MDAnalysis
from MDAnalysis.lib import distances, mdamath
import numpy as np

def read_cg(filename, n_cg, frame=-1):
    traj = MDAnalysis.Universe(filename)
    traj.trajectory[int(frame)]  # go to end of trajectory
    xyz = traj.atoms.positions
    _, types = np.unique(traj.atoms.types, return_inverse=True)
    box = traj.dimensions

    f = distances.transform_RtoS(xyz, box)

    f_cg = f[0::n_cg]
    types_cg = types[0::n_cg]
    for i in range(1, n_cg):
        vec = f[i::n_cg] - f[0::n_cg]
        vec -= np.round(vec)
        f_cg += vec
        types_cg += types[i::n_cg]

    f_cg -= np.round(f_cg)
    xyz_cg = distances.transform_StoR(f_cg, box) + 0.5 * box[:3]
    _, types_cg = np.unique(types_cg, return_inverse=True)

    return xyz_cg, box, types_cg

# Process the data

Mount Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Define the target set of simulations.

In [ ]:
target_dir = 'gru-cv-kmeans-v2'

Read the `gsd` files available on Google Drive.

In [ ]:
import glob, os

drive_prefix = '/content/drive/Shareddrives/Polymers-Data'
gsd_files = sorted(glob.glob(os.path.join(drive_prefix, 'trajectories', target_dir, '*.gsd')))
print(len(gsd_files))
filename = gsd_files[0]
print(filename)

200
/content/drive/Shareddrives/Polymers-Data/trajectories/gru-cv-kmeans-v2/Npol_500_Nmon_20_seq_AABAABBBABABBAAABAAA_run_0_traj_box_43.1_43.1_43.1_kT_0.5.gsd


## Calculate histograms

For each `gsd` file, load all the particle data and compute the representations of the local atomic environment.
Then save the result to Google Drive.

In [ ]:
import pickle
import tqdm

traj = MDAnalysis.Universe(gsd_files[0])

timesteps = np.arange(len(traj.trajectory))
# timesteps = [-1]

bins = 12  # don't change this!
for j, filename in enumerate(gsd_files):

    if os.path.isfile(filename.replace('.gsd', '_H.pkl')):
        continue

    print('processing {:} of {:}, {:s}'.format(j+1, len(gsd_files), filename))
    xyz, box, types = read_cg(filename, n_cg, frame=0)
    all_H = np.zeros([len(timesteps), len(xyz), n_species * bins, 3 * bins])

    complete = True
    for ts, timestep in tqdm.tqdm(enumerate(timesteps), total=len(timesteps)):
        try:
            xyz, box, types = read_cg(filename, n_cg, frame=timestep)
        except:
            complete = False
            break
        pairs, dists = distances.self_capped_distance(xyz, cutoff, box=box)

        def neighborhood(i):  # define neighborhood fetching function
            idx = np.argwhere(pairs == i)[:, 0]
            loc = np.unique(pairs[idx])
            f = distances.transform_RtoS(xyz[loc] - xyz[i], box)
            f -= np.round(f)
            r = distances.transform_StoR(f, box)
            t = types[loc]
            return r, t.reshape(-1, 1)

        # process the histograms
        idx = np.arange(xyz.shape[0])
        for k, i in enumerate(idx):
            r, t = neighborhood(i)
            data = torch.tensor(r, dtype=torch.float, requires_grad=False).to(device)
            output = model(data)
            z = output.to('cpu').detach().numpy()
            H = np.hstack([z[y, :].reshape(bins, bins) for y in range(3)])
            all_H[ts, k] = H

    if complete:
        with open(filename.replace('.gsd', '_H.pkl'), 'wb') as fid:
            pickle.dump(all_H, fid)

## Embed individual atomic environments in UMAP

This is an annoying workaround a `numba` bytecode incompatibility issue between different operating systems.
Usually we would just use `pickle` to serialize the `umap` object.

In [ ]:
import umap
import os

try:
    reducer
except:
    with open(os.path.join(drive_prefix, 'umap-reducers', 'umap-reducer-AB-1024-data.pkl'), 'rb') as fid:
        buffer = pickle.load(fid)

    reducer = umap.UMAP(n_components=3, n_neighbors=10, min_dist=0., random_state=0, verbose=False)
    reducer.fit(buffer['_raw_data'])
    reducer.embedding_ = buffer['embedding_']
    reducer.graph_ = buffer['graph_']

Embed the atomic environments from each frame into the same UMAP.

In [ ]:
hist_files = sorted(glob.glob(os.path.join(drive_prefix, 'trajectories', target_dir, '*_H.pkl')))
print('embedding {:} histograms from google drive'.format(len(hist_files)))

for j, filename in enumerate(hist_files):
    if os.path.isfile(filename.replace('_H.pkl', '_UMAP.pkl')):
        continue
    
    print('processing {:} of {:}, {:s}'.format(j+1, len(hist_files), filename))
    
    with open(filename, 'rb') as fid:
        all_H = pickle.load(fid)

    all_U = []
    for i, H in tqdm.tqdm(enumerate(all_H), total=len(all_H)):
        embedding = reducer.transform(H.reshape(H.shape[0], -1))
        all_U.append(embedding)

    with open(filename.replace('_H.pkl', '_UMAP.pkl'), 'wb') as fid:
        pickle.dump(all_U, fid)

embedding 200 histograms from google drive


Visualize the atomic environment space.

# Embed snapshots in histogram UMAP

Load the pre-existing histogram UMAP.
This is for entire frames.

In [ ]:
try:
    super_reducer
except:
    with open(os.path.join(drive_prefix, 'umap-reducers', 'hist-reducer-AB-1024-data.pkl'), 'rb') as fid:
        buffer = pickle.load(fid)
        
    super_reducer = umap.UMAP(n_components=2, n_neighbors=16, min_dist=1, random_state=0, verbose=False)
    super_reducer.fit(buffer['_raw_data'])
    super_reducer.embedding_ = buffer['embedding_']
    super_reducer.graph_ = buffer['graph_']

Define a Gaussian expansion histogram model to use for the embedding.

In [ ]:
import torch

if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"

device = torch.device(device)

hbins = 36
res = [hbins*0.5]*3
# ranges = np.vstack([embedding.min(axis=0), embedding.max(axis=0)]).T
ranges = np.array([[ 2.193795 , 10.049596 ], [ 2.736186 , 10.067611 ], [ 6.3948064,  9.304425 ]])
sigma = np.array([ranges[0, 1]/res[0], ranges[1, 1]/res[1], ranges[2, 1]/res[2]])

gh = GaussianHistogram(hbins, ranges, sigma, device=device)
gh.to(device)

Generate the histograms ("fingerprints") for each snapshot.

In [ ]:
fingerprints = []

for j, filename in tqdm.tqdm(enumerate(hist_files), total=len(hist_files)):
    
    with open(filename.replace('_H.pkl', '_UMAP.pkl'), 'rb') as fid:
        all_U = pickle.load(fid)

    for lam in all_U:

        X = torch.tensor(lam.T, device=device)
        yh = gh(X).to('cpu').detach().numpy()
        yh = [y.reshape(hbins, hbins).T for y in yh]
        yh = np.hstack([np.flipud(y) / y.sum() for y in yh])

        fingerprints.append(yh)

100%|██████████| 200/200 [00:07<00:00, 25.47it/s]


Embed the histogram of each frame in the global structure space.

In [ ]:
frame_embedding = super_reducer.transform(np.array(fingerprints).reshape(len(fingerprints), -1))

# Create a `CSV` file

In [ ]:
import pandas as pd

if '_id_' in hist_files[0]:
    keyword = '_id_'  # for random chains
else:
    keyword = '_seq_'  # for sequence-specified chains

frames = len(all_U)  # frames per trajectory
trajs = len(hist_files)  # number of trajectories

filename = [hist_files[int(i / frames)].replace(drive_prefix + '/', '').replace('_H.pkl', '.gsd') for i in range(frames * trajs)]
frame = [i % frames for i in range(frames * trajs)]
sequence = [f.split(keyword)[1].split('_run_')[0] for f in filename]
data = pd.DataFrame({'Filename': filename, 'Frame': frame, 'Sequence': sequence,
                     'Z0': frame_embedding[:, 0], 'Z1': frame_embedding[:, 1]})

# let's save it to the google drive folder:
destination_filename = os.path.join(os.path.join(drive_prefix, 'trajectories', target_dir), 'embeddings.csv')
data.to_csv(destination_filename)